[Maximum Swap](https://leetcode.com/problems/maximum-swap/)。一个数字，只允许对两位交换一次，求能得到的最大数。

思路：由高位往低位扫描，如果低位最大的数字大于高位，那么就做交换。可优化的地方有两处，一是使用哈希表记录数字的每一位，二是单位数字的取值范围是$[0,9]$。难点在于数字可能重复出现在不同位，那么只将最低位的数字换到高位。

In [ ]:
def maximumSwap(num: int) -> int:
    nums = list(map(int, str(num)))

    # 注意这里构造字典隐含一个技巧，同一个数字多次出现时，低位数字会覆盖高位数字
    tabel = {num: idx for idx, num in enumerate(nums)}

    for idx, cur_num in enumerate(nums):    # 遍历每一位
        for larger in range(9, cur_num, -1):    # 比当前位大的数字
            larger_idx = tabel.get(larger, None)
            if larger_idx and larger_idx > idx:    # 如果大数存在且出现在低位
                nums[larger_idx], nums[idx] = nums[idx], nums[larger_idx]
                return int(''.join(map(str, nums)))

    return num

[Maximum Subarray](https://leetcode.com/problems/maximum-subarray/)。求数组的最大连续和。

思路：贪心算法，当某一时刻的和小于$0$时，重新加。关键在于数组数字全负的情况，需要使用一个布尔变量来标记。

In [ ]:
def maxSubArray(nums) -> int:
    max_sum = nums[0]
    cur_sum = 0

    all_Neg = True

    for num in nums:
        if num > 0:
            all_Neg = False

        cur_sum += num
        max_sum = max(max_sum, cur_sum)

        if cur_sum < 0:
            cur_sum = 0

    if all_Neg:
        return max(nums)

    return max_sum

[Array of Doubled Pairs](https://leetcode.com/problems/array-of-doubled-pairs/)。给定一个偶数大小的数组，判断该数组能否组成数字对$(num1,num2)$，使得每一对都满足$num2=2*num1$。

思路：首先按照绝对值排序，由小往大取值。一个数$x$应该有一个$2x$与其成对，若$2x$不存在则返回false。另外一点就是需要对数字计数，避免重复使用。

In [2]:
def canReorderDoubled(A) -> bool:
    cnt = dict()
    for num in A:
        cnt.setdefault(num, 0)
        cnt[num] += 1

    A.sort(key=abs)

    for num in A:
        if cnt.get(num, 0) == 0:
            continue
        if cnt.get(2*num, 0) == 0:
            return False

        cnt[num] -= 1
        cnt[2*num] -= 1

    return True


canReorderDoubled([1, 2, 1, 2])

True

[Course Schedule III](https://leetcode.com/problems/course-schedule-iii/)。给出$n$门课程，每门课程有两个信息：(学习时长，截止日期)，求最多能完成多少门课程。

思路：可使用贪心算法获得最优解。优化目标是完成的课程数，那么易得在完成相同门数课程的条件下，学习时长越短越好；或者反过来说，在同样的学习时长下，完成的课程数越多越好。首先按照截止时间对课程排序，顺序选课，若能在截止日期前完成，则加入选课列表；若不能在截止日期内完成，则需要做取舍，将已选课中学习时长最长的课用当前课来代替。维护一个已选课程数组，该数组以学习时长排序，最长的排在最后边。

In [10]:
def scheduleCourse(courses) -> int:
    courses.sort(key=lambda x: x[1])
    schedule = list()    # 选课计划
    std_time = 0    # 总学习时间

    def BinSearch(l, x):
        '''
        寻找插入位置
        '''
        start, end = 0, len(l)
        while start < end:
            mid = (start+end)//2
            if l[mid] <= x:
                start = mid+1
            else:
                end = mid
        return start

    for course in courses:
        if course[0] > course[1]:    # 无法完成的课程
            continue

        if std_time+course[0] <= course[1]:    # 可无脑加入的课程
            idx = BinSearch(schedule, course[0])
            schedule.insert(idx, course[0])
            std_time += course[0]
        else:
            if course[0] < schedule[-1]:    # 替换时长最长的
                std_time += course[0]-schedule[-1]
                del schedule[-1]
                idx = BinSearch(schedule, course[0])
                schedule.insert(idx, course[0])

    return len(schedule)

2

[Wiggle Subsequence](https://leetcode.com/problems/wiggle-subsequence/)。曲折子序列，给定一数组，求其能构成曲折子数组的最大长度。

思路：扫描数组，若连续上升或连续下降，则找到区间的极值点；难点在于首个弯曲点的处理。

In [ ]:
def wiggleMaxLength(nums) -> int:
    if len(nums) < 2:
        return len(nums)

    res = 0
    pre_sign = None    # 之前的符号，True表正，False表负

    for idx in range(1, len(nums)):
        if nums[idx] > nums[idx-1] and (pre_sign is None or not pre_sign):
            pre_sign = True
            res += 1
        elif nums[idx] < nums[idx-1] and (pre_sign is None or pre_sign):
            pre_sign = False
            res += 1

    return res+1    # 加上首个数字

[Best Time to Buy and Sell Stock](https://leetcode.com/problems/best-time-to-buy-and-sell-stock/)。一数组代表股票的历史价格，允许买卖一次，求能获得的最大利润。

思路：贪心算法，遍历数组，维护一个最低价格与一个最大利润即可。

In [13]:
def maxProfit(prices) -> int:
    min_price = 0x7FFFFFFF
    max_profit = 0

    for price in prices:
        min_price = min(min_price, price)
        max_profit = max(max_profit, price-min_price)

    return max_profit

5

[Longest Word in Dictionary](https://leetcode.com/problems/longest-word-in-dictionary/)。给一个字典，求出该字典包含完整拼写过程的最长字符串。

思路：若字典中包含了某一字符串的完整拼写过程，说明该字符串的所有前缀都包含在字典里面。使用set进行暴力查找。

In [21]:
def longestWord(words) -> str:
    res = ''

    for s in words:
        n = len(s)
        i = 1

        while i <= n:
            if s[:i] in words:
                i += 1
            else:
                break

        if i == n+1:    # 完全匹配
            if len(s) > len(res):
                res = s
            elif len(s) == len(res):
                res = min(res, s)

    return res

[Longest Word in Dictionary through Deleting](https://leetcode.com/problems/longest-word-in-dictionary-through-deleting/)。给一个字符串与一个字典，找出在只能删除字符的情况下，在字典中能匹配到的最长的字符串。当多个匹配字符串长度相等时，返回字母序靠前的。

思路：暴力搜索，维护一个匹配值，当新匹配的字典值长度大于之前的，又或者等于之前的但是字母序在前时，这两种情况需要替换。Python可以直接比较字典序。

In [20]:
def findLongestWord(s: str, d) -> str:
    res = ''
    n_s = len(s)

    for word in d:
        i = j = 0
        n_word = len(word)

        while i < n_s and j < n_word:    # 顺序匹配
            if s[i] == word[j]:
                i += 1
                j += 1
            else:
                i += 1

        if j == n_word:    # 完全匹配
            if len(word) > len(res):
                res = word
            elif len(word) == len(res):
                res = min(res, word)

    return res

'a'